In [111]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Suffix, value
from pyomo.opt import SolverFactory
import pandas as pd

# 1. read all input data from files

relevant tsv-files are:
- load
- duration
- availability
- tech_data

In [112]:
load = pd.read_csv("load.tsv",  sep="\s+", names=["value"] )
load = load.T
load #[MW]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,82115,73169,68729,63442,60430,57013,52048,48701,43981,40498


In [113]:
tech_data = pd.read_csv("tech_data.tsv", sep="\s+", header=0, index_col=False, skiprows=[1], decimal=".") #workaround bc read csv shifts columnnames
tech_data.set_index("tech", inplace=True)
tech_data

,Cap,ETA_EL,Fuel_P,c_var_other,EMF
tech,,,,,
CCGT,29555,0.54,12.8,1.5,0.2048
GT_GasOil,4400,0.28,12.8,1.5,0.2048
Hydro,5256,1.00,0.0,1.5,0.0000
Coal,22458,0.42,7.4,2.6,0.3420
Lignite,21067,0.37,3.4,3.0,0.3996
Nuclear,8114,0.33,1.8,0.7,0.0000
Wind,61114,1.00,0.0,1.4,0.0000
Solar,46471,1.00,0.0,1.0,0.0000


In [114]:
availability = pd.read_csv("availability.tsv", sep="\s+")
availability #[MW]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,19629,19629,19629,19629,19629,19629,19629,19629,19629,19629
GT_GasOil,2980,2980,2980,2980,2980,2980,2980,2980,2980,2980
Hydro,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012
Coal,19564,19564,19564,19564,19564,19564,19564,19564,19564,19564
Lignite,17687,17687,17687,17687,17687,17687,17687,17687,17687,17687
Nuclear,7610,7610,7610,7610,7610,7610,7610,7610,7610,7610
Wind,11480,9858,11332,14498,20590,21012,6314,16072,21238,12054
Solar,10538,2078,3920,7641,6970,0,14810,1307,0,0


In [115]:
duration = pd.read_csv("duration.tsv", sep="\s+", names=["value"])
duration = duration.T
duration #[h]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,102,962,962,962,962,962,962,962,962,962


# 2. set up calculations
if thats necessary.

In [116]:
tech_data["costs_el_no_co2"] = tech_data.Fuel_P / tech_data.ETA_EL + tech_data.c_var_other
tech_data.costs_el_no_co2 #[€/MWh_el]

tech
CCGT         25.203704
GT_GasOil    47.214286
Hydro         1.500000
Coal         20.219048
Lignite      12.189189
Nuclear       6.154545
Wind          1.400000
Solar         1.000000
Name: costs_el_no_co2, dtype: float64

In [117]:
tech_data["emissions_el"] = tech_data.EMF / tech_data.ETA_EL
tech_data.emissions_el #[t/MWh_el]

tech
CCGT         0.379259
GT_GasOil    0.731429
Hydro        0.000000
Coal         0.814286
Lignite      1.080000
Nuclear      0.000000
Wind         0.000000
Solar        0.000000
Name: emissions_el, dtype: float64

In [118]:
co2_price = 50
tech_data["costs_el_w_co2"] = (tech_data.Fuel_P / tech_data.ETA_EL) + tech_data.c_var_other + (co2_price * tech_data.EMF / tech_data.ETA_EL)
tech_data #[€/MWh_el]

,Cap,ETA_EL,Fuel_P,c_var_other,EMF,costs_el_no_co2,emissions_el,costs_el_w_co2
tech,,,,,,,,
CCGT,29555,0.54,12.8,1.5,0.2048,25.203704,0.379259,44.166667
GT_GasOil,4400,0.28,12.8,1.5,0.2048,47.214286,0.731429,83.785714
Hydro,5256,1.00,0.0,1.5,0.0000,1.500000,0.000000,1.500000
Coal,22458,0.42,7.4,2.6,0.3420,20.219048,0.814286,60.933333
Lignite,21067,0.37,3.4,3.0,0.3996,12.189189,1.080000,66.189189
Nuclear,8114,0.33,1.8,0.7,0.0000,6.154545,0.000000,6.154545
Wind,61114,1.00,0.0,1.4,0.0000,1.400000,0.000000,1.400000
Solar,46471,1.00,0.0,1.0,0.0000,1.000000,0.000000,1.000000


# 3. configuration of model

defined function to solve each timestep

In [185]:
def model_solve (timestep, co2=False):
    model = ConcreteModel()

    model.x = Var(tech_data.index.values, domain=NonNegativeReals) #dispatch variable

    model.balance = Constraint(expr = sum(model.x[gen] for gen in tech_data.index.values) == load.loc[:, timestep].value) # meet demand

    def avail_cap(model,gen):
        return model.x[gen] <= availability.loc[gen, timestep] # availability

    model.cap_limits = Constraint(tech_data.index.values, rule=avail_cap)
    
    if co2:
        model.objective = Objective(expr = sum([tech_data.costs_el_w_co2[gen]*model.x[gen] for gen in tech_data.index.values])) #minimizing €/h load dispatch: missing static variable duration, but it has no influence on optimization of each timestep
    else:
        model.objective = Objective(expr = sum([tech_data.costs_el_no_co2[gen]*model.x[gen] for gen in tech_data.index.values])) #minimizing €/h load dispatch: missing static variable duration, but it has no influence on optimization of each timestep

    opt = SolverFactory("glpk")

    model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

    results = opt.solve(model,suffixes=["dual"]) # model.x = dispatch in [MW]

    #results.write()
    return model, results

# 4. Post modeling

## 4.1 create dispatch dataframe from result
-> Toggle co2_price!

In [186]:
co2_price = True # toggle co2 False and True for co2-price
result_df = pd.DataFrame()
for t in load.columns.values:
    model, results = model_solve(t, co2_price)     
    for gen in model.x: 
        result_df.loc[gen,t] = model.x[gen].value
result_df = result_df.reindex(tech_data.index) #same order as tech_data
result_df #[MW]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
tech,,,,,,,,,,
CCGT,19629.0,19629.0,19629.0,19629.0,19629.0,19629.0,19302.0,19629.0,11121.0,16822.0
GT_GasOil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hydro,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0,4012.0
Coal,19564.0,19564.0,19564.0,10052.0,1619.0,4750.0,0.0,71.0,0.0,0.0
Lignite,9282.0,10418.0,2662.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nuclear,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0,7610.0
Wind,11480.0,9858.0,11332.0,14498.0,20590.0,21012.0,6314.0,16072.0,21238.0,12054.0
Solar,10538.0,2078.0,3920.0,7641.0,6970.0,0.0,14810.0,1307.0,0.0,0.0


In [124]:
result_energy_df = result_df.mul(duration.values, axis=1) 
result_energy_df #[MWh_el]
    

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
tech,,,,,,,,,,
CCGT,2002158.0,18883098.0,18883098.0,18883098.0,18883098.0,18883098.0,18568524.0,18883098.0,10698402.0,16182764.0
GT_GasOil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hydro,409224.0,3859544.0,3859544.0,3859544.0,3859544.0,3859544.0,3859544.0,3859544.0,3859544.0,3859544.0
Coal,1995528.0,18820568.0,18820568.0,9670024.0,1557478.0,4569500.0,0.0,68302.0,0.0,0.0
Lignite,946764.0,10022116.0,2560844.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nuclear,776220.0,7320820.0,7320820.0,7320820.0,7320820.0,7320820.0,7320820.0,7320820.0,7320820.0,7320820.0
Wind,1170960.0,9483396.0,10901384.0,13947076.0,19807580.0,20213544.0,6074068.0,15461264.0,20430956.0,11595948.0
Solar,1074876.0,1999036.0,3771040.0,7350642.0,6705140.0,0.0,14247220.0,1257334.0,0.0,0.0


## 4.2 calculate co2 emissions

In [131]:
co2_df = result_energy_df.mul(tech_data.emissions_el, axis=0) #multiply with emission factors
co2_df = co2_df * 10e-6
co2_df #[Mt]
#TODO: Excel export

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
tech,,,,,,,,,,
CCGT,7.593370,71.615898,71.615898,71.615898,71.615898,71.615898,70.422847,71.615898,40.57468,61.374631
GT_GasOil,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Hydro,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Coal,16.249299,153.253197,153.253197,78.741624,12.682321,37.208786,0.000000,0.556173,0.00000,0.000000
Lignite,10.225051,108.238853,27.657115,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Nuclear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Wind,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
Solar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [132]:
co2_df.sum() # co2 emissions in each timestep

t1      34.067720
t2     333.107947
t3     252.526209
t4     150.357522
t5      84.298218
t6     108.824683
t7      70.422847
t8      72.172071
t9      40.574680
t10     61.374631
dtype: float64

In [133]:
co2_df.sum().sum() # annual co2 emissions in [Mt] #TODO

1207.7265286010581

## 4.3 calculate electricity price

In [137]:
price_df = result_df > 0
if co2_price:
    price_df = price_df.mul(tech_data.costs_el_w_co2, axis=0)
else:
    price_df = price_df.mul(tech_data.costs_el_no_co2, axis=0)
price_df #price for each gen each timestep [€/MWh_el]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
tech,,,,,,,,,,
CCGT,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667,44.166667
GT_GasOil,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Hydro,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000
Coal,60.933333,60.933333,60.933333,60.933333,60.933333,60.933333,0.000000,60.933333,0.000000,0.000000
Lignite,66.189189,66.189189,66.189189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Nuclear,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545,6.154545
Wind,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000,1.400000
Solar,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000


In [138]:
elec_price = pd.DataFrame(price_df.max(axis=0), columns=["value"])
elec_price = elec_price.T 
elec_price #marketprice each timestep [€/MWh_el] #TODO

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,66.189189,66.189189,66.189189,60.933333,60.933333,60.933333,44.166667,60.933333,44.166667,44.166667


In [144]:
timestep_price = elec_price.mul(load, axis=1).mul(duration, axis=1)
timestep_price #[€]

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
value,5.543828e+08,4.658963e+09,4.376250e+09,3.718835e+09,3.542278e+09,3.341980e+09,2.211433e+09,2.854749e+09,1.868679e+09,1.720693e+09


In [164]:
annual_price = timestep_price.sum(axis=1) 
annual_price #[€]

value    2.884824e+10
dtype: float64

In [165]:
annual_energy = load.mul(duration, axis=1).sum(axis=1) #sum of every energy amount of each time step #moon
annual_energy

value    497082312
dtype: int64

In [167]:
weighted_annual_price = annual_price / annual_energy #average electricity price weighted by amount of energy
weighted_annual_price[0] #[€/MWh_el] #TODO

58.03514137332832

## 4.4 calculate contribution

In [174]:
annual_revenue = result_energy_df.mul(elec_price.values, axis=1).sum(axis=1)
if co2_price:
    annual_var_cost = result_energy_df.mul(price_df).sum(axis=1)
annual_contribution = annual_revenue - annual_var_cost
annual_contribution [€] #TODO
pd.to_csv()

tech
CCGT         2.142226e+09
GT_GasOil    0.000000e+00
Hydro        1.937378e+09
Coal         2.083246e+08
Lignite      0.000000e+00
Nuclear      3.364548e+09
Wind         7.159398e+09
Solar        1.978988e+09
dtype: float64

# shadow price

In [176]:
model.dual[model.balance]

44.1666666666667

In [182]:
model.dual[model.cap_limits[0]]

KeyError: "Index '0' is not valid for indexed component 'cap_limits'"